# Segmentating and Clustering Neigborhoods in Toronto
## Question 1 - Webscrape data of Borough and Neighborhood of Toronto
Luis Lamas - May 2021

In [22]:
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import requests
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN 
import matplotlib.cm as cm
import matplotlib.colors as colors

pd.set_option('display.max_rows', None)

## Scrape Borough values for Toronto
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data  = requests.get(url).text 
soup = BeautifulSoup(data,"html5lib")  


In [3]:
df_Toronto = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])

table = soup.find('table')
for row in table.find_all('tr'): # in html table row is represented by the tag <tr>
    # Get all columns in each row.
    cols = row.find_all('p') # in html a column is represented by the tag <td>
    for col in cols:
        col.get_text(strip=True)
        try:
            aux_text = [text for text in col.stripped_strings]
            zip_code = aux_text[0]
            borough = aux_text[1]
            if len(aux_text) == 3:
                neighborhood = aux_text[2]
                neighborhood = neighborhood.replace('(', '').replace(')', '').replace('/', ',').strip(' ')
#                neighborhood = neighborhood.replace(')', '')
            elif len(aux_text) > 3:
                neighborhood = aux_text[2:]
                neighborhood = ' '.join(neighborhood)
                neighborhood = neighborhood.replace('(', '').replace(')', '').replace('/', ',').strip(' ')
                #neighborhood = neighborhood.replace(')', '')
            else:
                neighborhood = 'N/A'
            
        except:
            pass
        #print("{}, {}, {}".format(zip_code, borough, neighborhood))
        df_Toronto = df_Toronto.append({"PostalCode":zip_code, "Borough":borough, "Neighborhood":neighborhood}, ignore_index=True)
        
df_Toronto.drop(df_Toronto[df_Toronto.Borough == 'Not assigned'].index, inplace=True)        
df_Toronto.head(103)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern , Rouge"
11,M3B,North York,Don Mills North
12,M4B,East York,"Parkview Hill , Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District , Ryerson"


In [4]:
df_Toronto.shape

(103, 3)

## Add Geospatial Data to DataFrame

In [5]:
df_geo = pd.read_csv('Geospatial_Coordinates.csv')
df_geo

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [6]:
df_Toronto2 = df_Toronto.copy()

df_Toronto2.columns = ['Postal Code', 'Borough', 'Neighborhood']
df_Toronto2 = pd.merge(df_Toronto2, df_geo, on="Postal Code")
df_Toronto2

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District , Ryerson",43.657162,-79.378937


In [7]:
df_Toronto2['Borough'].unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

## Clustering analysis

## Create Map of Toronto using Folium

In [8]:
# create map of Toronto using latitude and longitude values
import folium # map rendering library
latitude, longitude = 43.6532, -79.3832
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_Toronto2['Latitude'], df_Toronto2['Longitude'], df_Toronto2['Borough'], df_Toronto2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Retrieve data from Foursquare

In [9]:
CLIENT_ID = '4JSGKADZIJ1DKEXHDWZ5GNHJ44H2PTS0ONXGS3SI2PI3S3UQ' # your Foursquare ID
CLIENT_SECRET = 'IX5K5P4XBYKILUP5RRPPR10Q05YVJPYXJNGQFYR0E3CA02P0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [10]:
#Function to retrive venues near a list of Neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
toronto_venues = getNearbyVenues(names=df_Toronto2['Neighborhood'],
                                   latitudes=df_Toronto2['Latitude'],
                                   longitudes=df_Toronto2['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park , Harbourfront
Lawrence Manor , Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern , Rouge
Don Mills  North
Parkview Hill , Woodbine Gardens
Garden District , Ryerson
Glencairn
West Deane Park , Princess Gardens , Martin Grove , Islington , Cloverdale
Rouge Hill , Port Union , Highland Creek
Don Mills  South  Flemingdon Park
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate , Bloordale Gardens , Old Burnhamthorpe , Markland Wood
Guildwood , Morningside , West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor , Wilson Heights , Downsview North
Thorncliffe Park
Richmond , Adelaide , King
Dufferin , Dovercourt Village
Scarborough Village
Fairview , Henry Farm , Oriole
Northwood Park , York University
East Toronto  The Danforth East
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
Kennedy Park , Ionview , Eas

In [12]:
toronto_venues


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,TTC stop #8380,43.752672,-79.326351,Bus Stop
2,Parkwoods,43.753259,-79.329656,Sun Life,43.754760,-79.332783,Construction & Landscaping
3,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
5,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
6,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
7,Victoria Village,43.725882,-79.315572,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection
8,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place
9,"Regent Park , Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery


In [13]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 269 uniques categories.


## Analyze Neighborhoods

In [14]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.shape

(102, 269)

Create a DataFrame with the 10 top_venues at each neighborhoods

In [81]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Skating Rink,Women's Store,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,"Alderwood , Long Branch",Pizza Place,Coffee Shop,Pub,Pool,Sandwich Place,Playground,Gym,Drugstore,Donut Shop,Doner Restaurant
2,"Bathurst Manor , Wilson Heights , Downsview North",Coffee Shop,Bank,Pharmacy,Fried Chicken Joint,Shopping Mall,Bridal Shop,Diner,Sandwich Place,Middle Eastern Restaurant,Deli / Bodega
3,Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Women's Store
4,"Bedford Park , Lawrence Manor East",Italian Restaurant,Sandwich Place,Coffee Shop,Restaurant,Comfort Food Restaurant,Hobby Shop,Spa,Juice Bar,Fast Food Restaurant,Butcher


In [82]:
neighborhoods_venues_sorted.shape

(102, 11)

## Cluster Neighborhoods

In [83]:
# set number of clusters - kmeans
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 4, 0, 2, 0, 0, 0, 1, 0, 0, 2, 2])

In [84]:
epsilon = 0.3
minimumSamples = 3
db = DBSCAN(eps=epsilon, min_samples=minimumSamples).fit(toronto_grouped_clustering)
labels = db.labels_
labels

array([-1, -1,  0, -1,  0,  0, -1,  0,  0, -1, -1,  0, -1,  0, -1,  0, -1,
       -1,  0,  0, -1, -1, -1,  0, -1, -1, -1, -1, -1,  0,  1,  0,  0,  0,
       -1,  0, -1, -1, -1,  0,  0, -1, -1, -1, -1,  0, -1, -1,  0, -1, -1,
       -1,  0,  0, -1, -1,  0, -1,  0, -1,  0, -1, -1, -1,  0,  0, -1, -1,
        0,  0, -1, -1, -1,  0, -1, -1, -1,  0,  0, -1,  0,  0,  0,  0, -1,
        0, -1,  0,  0,  0, -1, -1, -1,  1, -1,  0, -1, -1, -1, -1, -1,  1],
      dtype=int64)

In [85]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_Toronto2

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.dropna(inplace=True)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Park,Construction & Landscaping,Bus Stop,Food & Drink Shop,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Pizza Place,Coffee Shop,Intersection,Portuguese Restaurant,Hockey Arena,Eastern European Restaurant,Electronics Store,Dumpling Restaurant,Drugstore,Escape Room
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Café,Pub,Bakery,Breakfast Spot,Theater,Farmers Market,Historic Site,Beer Store
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763,0,Clothing Store,Furniture / Home Store,Vietnamese Restaurant,Miscellaneous Shop,Coffee Shop,Women's Store,Boutique,Accessories Store,Gift Shop,Electronics Store
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Bar,Beer Bar,Spa,Smoothie Shop,Sandwich Place,Burrito Place,Café,Portuguese Restaurant


In [86]:
toronto_merged['Cluster Labels'].astype(int)

0      0
1      0
2      0
3      0
4      0
5      0
6      0
7      0
8      0
9      0
10     0
11     4
12     0
13     0
14     0
15     0
16     3
17     0
18     0
19     0
20     0
21     2
22     0
23     0
24     0
25     0
26     0
27     0
28     0
29     0
30     0
31     0
32     3
33     0
34     0
35     2
36     0
37     0
38     0
39     0
40     0
41     0
42     0
43     0
44     0
45     2
46     0
47     0
48     0
49     0
50     0
51     0
52     1
53     0
54     0
55     0
56     0
57     0
58     0
59     0
60     0
61     0
62     0
63     0
64     2
65     0
66     2
67     0
68     0
69     0
70     0
71     0
72     0
73     0
74     0
75     0
76     0
77     0
78     0
79     0
80     0
81     0
82     0
83     3
84     0
85     0
86     0
87     0
88     0
89     0
90     0
91     2
92     0
93     0
94     0
96     0
97     0
98     0
99     0
100    0
101    0
102    0
Name: Cluster Labels, dtype: int32

In [87]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    #cluster = int(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [89]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Park,Construction & Landscaping,Bus Stop,Food & Drink Shop,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
1,North York,0,Pizza Place,Coffee Shop,Intersection,Portuguese Restaurant,Hockey Arena,Eastern European Restaurant,Electronics Store,Dumpling Restaurant,Drugstore,Escape Room
2,Downtown Toronto,0,Coffee Shop,Park,Café,Pub,Bakery,Breakfast Spot,Theater,Farmers Market,Historic Site,Beer Store
3,North York,0,Clothing Store,Furniture / Home Store,Vietnamese Restaurant,Miscellaneous Shop,Coffee Shop,Women's Store,Boutique,Accessories Store,Gift Shop,Electronics Store
4,Queen's Park,0,Coffee Shop,Sushi Restaurant,Bar,Beer Bar,Spa,Smoothie Shop,Sandwich Place,Burrito Place,Café,Portuguese Restaurant
5,Etobicoke,0,Pizza Place,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
6,Scarborough,0,Fast Food Restaurant,Dessert Shop,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
7,North York,0,Athletics & Sports,Caribbean Restaurant,Japanese Restaurant,Café,Gym,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
8,East York,0,Pizza Place,Gastropub,Gym / Fitness Center,Café,Breakfast Spot,Pet Store,Flea Market,Bank,Intersection,Athletics & Sports
9,Downtown Toronto,0,Clothing Store,Coffee Shop,Italian Restaurant,Bubble Tea Shop,Cosmetics Shop,Café,Japanese Restaurant,Hotel,Middle Eastern Restaurant,Fast Food Restaurant
